In [11]:
import joblib
import pandas as pd
import sqlite3 as sq

from itertools import chain

In [12]:
def format_anime_base(anime):
    return pd.DataFrame([
        dict(
            anime_id = anime['mal_id'],
            url = anime['url'],
            image_jpg = anime['images'].get('jpg', dict()).get('image_url'),
            image_jpg_small = anime['images'].get('jpg', dict()).get('small_image_url'),
            image_jpg_large = anime['images'].get('jpg', dict()).get('large_image_url'),
            image_webp = anime['images'].get('webp', dict()).get('image_url'),
            image_webp_small = anime['images'].get('webp', dict()).get('small_image_url'),
            image_webp_large = anime['images'].get('webp', dict()).get('large_image_url'),
            approved = anime['approved'],
            type = anime['type'],
            source = anime['source'],
            episodes = anime['episodes'],
            status = anime['status'],
            airing = anime['airing'],
            aired_from = anime['aired'].get('from'),
            aired_to = anime['aired'].get('to'),
            duration = anime['duration'],
            rating = anime['rating'],
            score = anime['score'],
            scored_by = anime['scored_by'],
            rank = anime['rank'],
            popularity = anime['popularity'],
            season = anime['season'],
            year = anime['year']
        )
    ])


def format_X(anime, key):
    return pd\
        .DataFrame(anime[f'{key}s'])\
        .rename(dict(mal_id=f'{key}_id'), axis=1)


def format_anime_X(anime, key):
    return pd\
        .concat([
            pd.DataFrame(columns=['mal_id', 'type', 'name', 'url']),
            pd.DataFrame(anime[f'{key}s'])\
        ])\
        .assign(anime_id = anime['mal_id'])\
        [['anime_id', f'mal_id']]\
        .rename(dict(mal_id=f'{key}_id'), axis=1)



def format_anime_title(anime):
    return pd\
        .DataFrame(anime['titles'])\
        .assign(anime_id = anime['mal_id'])\
        [['anime_id', 'title', 'type']]


def format_anime(anime,i):
    if not i % 100:
        print(i)

    return dict(
        df_anime             = format_anime_base(anime),
        df_producer          = format_X(anime, 'producer'),
        df_licensor          = format_X(anime, 'licensor'),
        df_studio            = format_X(anime, 'studio'),
        df_genre             = format_X(anime, 'genre'),
        df_theme             = format_X(anime, 'theme'),
        df_demographic       = format_X(anime, 'demographic'),
        df_anime_producer    = format_anime_X(anime, 'producer'),
        df_anime_licensor    = format_anime_X(anime, 'licensor'),
        df_anime_studio      = format_anime_X(anime, 'studio'),
        df_anime_genre       = format_anime_X(anime, 'genre'),
        df_anime_theme       = format_anime_X(anime, 'theme'),
        df_anime_demographic = format_anime_X(anime, 'demographic'),
        df_anime_title       = format_anime_title(anime)
    )


def get_schema(table, db='animedb.sqlite'):
    conn = sq.connect(db)
    cursor = conn.cursor()
    schema = cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' and name='{table}';").fetchall()[0][0]
    conn.close()

    return schema


def update_db(df, table, duplicate_cols, db='animedb.sqlite'):
    print('select old')
    with sq.connect(db) as conn:
        df = pd\
            .concat([
                df,
                pd.read_sql(f"select * from {table}", conn)
            ]).drop_duplicates(subset=duplicate_cols)
    
    print('get schema')
    schema = get_schema(table, db)

    print('drop table')
    with sq.connect(db) as conn:
        cursor = conn.cursor()
        cursor.execute(f'DROP TABLE {table}')
        cursor.close()
        conn.commit()

    print('create table with schema')
    with sq.connect(db) as conn:
        cursor = conn.cursor()
        cursor.execute(schema)
        cursor.close()
        conn.commit()

    print('append data')
    with sq.connect(db) as conn:
        df.to_sql(table, conn, if_exists='append', index=False)
        conn.commit()


In [13]:
db_name = 'animedb6.sqlite'
tables = [
    'anime',
    'producer',
    'licensor',
    'studio',
    'genre',
    'theme',
    'demographic',
    'anime_producer',
    'anime_licensor',
    'anime_studio',
    'anime_genre',
    'anime_theme',
    'anime_demographic',
    'anime_title'
]

In [14]:
conn = sq.connect(db_name)
cursor = conn.cursor()
schemas = cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table';").fetchall()
conn.close()
schemas

In [15]:
conn = sq.connect(db_name)
df = pd.concat([
    pd.read_sql(f"select count(*) as count_{table} from {table}", conn)
    for table in tables
], axis=1)
conn.close()
df.T

In [16]:
responses = joblib.load('anime_responses.pkl')
response_list = list(chain(*[r['data'] for r in responses]))

In [17]:
animesf = [format_anime(anime,i) for i,anime in enumerate(response_list[:10])]

In [18]:
df_anime = pd.concat([anime['df_anime'] for anime in animesf])
df_producer = pd.concat([anime['df_producer'] for anime in animesf]).drop_duplicates()
df_licensor = pd.concat([anime['df_licensor'] for anime in animesf]).drop_duplicates()
df_studio = pd.concat([anime['df_studio'] for anime in animesf]).drop_duplicates()
df_genre = pd.concat([anime['df_genre'] for anime in animesf]).drop_duplicates()
df_theme = pd.concat([anime['df_theme'] for anime in animesf]).drop_duplicates()
df_demographic = pd.concat([anime['df_demographic'] for anime in animesf]).drop_duplicates()
df_anime_producer = pd.concat([anime['df_anime_producer'] for anime in animesf])
df_anime_licensor = pd.concat([anime['df_anime_licensor'] for anime in animesf])
df_anime_studio = pd.concat([anime['df_anime_studio'] for anime in animesf])
df_anime_genre = pd.concat([anime['df_anime_genre'] for anime in animesf])
df_anime_theme = pd.concat([anime['df_anime_theme'] for anime in animesf])
df_anime_demographic = pd.concat([anime['df_anime_demographic'] for anime in animesf])
df_anime_title = pd.concat([anime['df_anime_title'] for anime in animesf])

In [19]:
conn = sq.connect(db_name)
dddd = pd.read_sql(f"select * from anime", conn)
conn.close()
dddd

In [20]:
update_db(df_anime,             'anime',             ['anime_id'],                   db=db_name)
update_db(df_producer,          'producer',          ['producer_id'],                db=db_name)
update_db(df_licensor,          'licensor',          ['licensor_id'],                db=db_name)
update_db(df_studio,            'studio',            ['studio_id'],                  db=db_name)
update_db(df_genre,             'genre',             ['genre_id'],                   db=db_name)
update_db(df_theme,             'theme',             ['theme_id'],                   db=db_name)
update_db(df_demographic,       'demographic',       ['demographic_id'],             db=db_name)
update_db(df_anime_producer,    'anime_producer',    ['anime_id', 'producer_id'],    db=db_name)
update_db(df_anime_licensor,    'anime_licensor',    ['anime_id', 'licensor_id'],    db=db_name)
update_db(df_anime_studio,      'anime_studio',      ['anime_id', 'studio_id'],      db=db_name)
update_db(df_anime_genre,       'anime_genre',       ['anime_id', 'genre_id'],       db=db_name)
update_db(df_anime_theme,       'anime_theme',       ['anime_id', 'theme_id'],       db=db_name)
update_db(df_anime_demographic, 'anime_demographic', ['anime_id', 'demographic_id'], db=db_name)
update_db(df_anime_title,       'anime_title',       ['anime_id', 'title'],          db=db_name)

In [21]:
conn = sq.connect(db_name)
cursor = conn.cursor()
schemas = cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table';").fetchall()
conn.close()
schemas

In [22]:
conn = sq.connect(db_name)
df = pd.concat([
    pd.read_sql(f"select count(*) as count_{table} from {table}", conn)
    for table in tables
], axis=1)
conn.close()
df.T